In [63]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os
import numpy as np

In [ ]:
metadata = pd.read_csv('D:/data/MCF10A commons metadata.csv',index_col=0)
metadata.Conc = metadata.Conc.apply(lambda x: "{:.1e}".format(x))

condition_vc = metadata.groupby(['DrugName','time','Conc','Cluster']).well.count()
# This is a very cool way of using multiindex, first get the by level counts and reset index to align with the orginal one then do the element-wise div to get the frequency
level_sum = condition_vc.groupby(['DrugName','time','Conc']).transform('sum')
condition_weight = condition_vc.div(level_sum)

df_fc = pd.read_excel('D:/data/MCF10A commons report.xlsx',index_col=0)
# print(metadata.head())
print(df_fc.head())

In [13]:
# make heatmap
df_fc['condition'] = df_fc.DrugName + '_' + df_fc.Dose.astype(str)+ '_' + df_fc.Time + '_' + df_fc.Cluster
df_summary = df_fc.pivot(values='logFC',columns='condition').transpose()
offset = []
for i,tag in enumerate(['Drug','Dose','Time']):
    metadata = [x.split('_')[i] for x in df_summary.index]
    df_summary.insert(0,tag,metadata)
    offset+=['']
df_summary = df_summary.transpose()
for i, tag in enumerate(['Location','Marker']):
    metadata = [x.split('_')[i] for x in df_summary.index[3:]]
    df_summary.insert(0,tag,offset+metadata)
df_summary.to_csv('d:/data/MCF10A drug response heatmap.csv')

In [65]:
# Make series of drug by marker line plots over time and clusters.
os.chdir('D:/data')
sns.set(font_scale=2)
for drug in df_fc.DrugName.unique():
    if drug=='DMSO':
        continue
    sub_df_fc = df_fc[df_fc.DrugName==drug]
    sub_df_fc.reset_index(inplace=True)
    sub_df_fc = sub_df_fc.sort_values(['Dose','index'])
    sub_df_fc.Dose = np.log10(sub_df_fc.Dose)
    g = sns.FacetGrid(sub_df_fc,col_wrap=10,col='index',hue='Time',hue_order=['24h','48h','72h'], height=5)
    g = (g.map(plt.plot,'Dose','logFC', marker=".", linewidth=5)
         .set_axis_labels("log10 Dose", "Dose")
         .set_titles(drug + " {col_name} ", weight='bold')
         .fig.subplots_adjust(wspace=.05, hspace=.1))
    plt.legend(bbox_to_anchor = (1.1,0.6))
    plt.savefig(drug + ' marker logFC over time.png')
    plt.close()